In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

# API Bağlantısının Yapılması

In [ ]:
!pip install tweepy

In [ ]:
import tweepy, codecs


consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
api.update_status("Hello from Python for Udemy Data Science and Machine Learning Course")

# Twitter'dan Veri Çekmek

In [ ]:
mvk = api.me()

In [ ]:
mvk.screen_name

In [ ]:
mvk.followers_count

In [ ]:
mvk.friends

In [ ]:
for friend in mvk.friends(count = 20):
    print(friend.screen_name)

In [ ]:
dir(mvk)

In [ ]:
#kullanıcı temel bilgileri

In [ ]:
user = api.get_user(id = "murat_aksit")

In [ ]:
dir(user)

In [ ]:
user.screen_name

In [ ]:
user.followers_count

In [ ]:
?user

In [ ]:
dir(user)

In [ ]:
user.profile_image_url

In [ ]:
# hometimeline

In [ ]:
public_tweets = api.home_timeline(count = 10)

In [ ]:
public_tweets

In [ ]:
for tweet in public_tweets:
    print(tweet.text)

In [ ]:
#user timeline

In [ ]:
name = "murat_aksit"
tweet_count = 10

user_timetime = api.user_timeline(id = name, count = tweet_count)

for i in user_timetime:
    print(i.text)

In [ ]:
#retweet edilen tweetler

In [ ]:
retweets = api.retweets_of_me(count = 3)
for retweet in retweets:
    print(retweet.text)

In [ ]:
#hashtag

In [ ]:
results = api.search(q = "#pazartesi", 
                     lang = "tr", 
                     result_type = "recent", 
                     count = 100)

In [ ]:
results

In [ ]:
for retweet in results:
    print(retweet.text)

In [ ]:
#dataframe cevirmek

In [ ]:
def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    import pandas as pd
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    
    return data_set

In [ ]:
data = tweets_df(results)

In [ ]:
data["text"]

In [ ]:
data.to_csv("data_twitter.csv")

# Profil Analizi

In [ ]:
#temel bilgiler

In [ ]:
mvk = api.get_user("mvahitkeskin")

In [ ]:
mvk.name

In [ ]:
mvk.id

In [ ]:
mvk.url

In [ ]:
mvk.verified

In [ ]:
mvk.screen_name

In [ ]:
mvk.location

In [ ]:
mvk.statuses_count

In [ ]:
mvk.followers_count

In [ ]:
mvk.favourites_count

In [ ]:
mvk.friends_count

In [ ]:
tweets = api.user_timeline(id = "mvahitkeskin")

In [ ]:
for i in tweets:
    print(i.text)

In [ ]:
def timeline_df(tweets):
    idler = [tweet.id for tweet  in tweets]
    import pandas as pd
    df = pd.DataFrame(idler, columns = ["id"])
    
    df["created_at"] = [tweet.created_at for tweet in tweets]
    df["text"] = [tweet.text for tweet in tweets]
    df["favorite_count"] = [tweet.favorite_count for tweet in tweets]
    df["retweet_count"] = [tweet.retweet_count for tweet in tweets]
    df["source"] = [tweet.source for kisi in tweets]
    
    return df

In [ ]:
timeline_df(tweets)

In [ ]:
def  timeline_df(tweets):
    import pandas as pd
    df = pd.DataFrame()
    df['id'] = list(map(lambda tweet: tweet.id, tweets))
    df['created_at'] = list(map(lambda tweet: tweet.created_at, tweets))
    df['text'] = list(map(lambda tweet: tweet.text, tweets))
    df['favorite_count'] = list(map(lambda tweet: tweet.favorite_count, tweets))
    df['retweet_count'] = list(map(lambda tweet: tweet.retweet_count, tweets))
    df['source'] = list(map(lambda tweet: tweet.source, tweets))
    return df

In [ ]:
tweets = api.user_timeline(id = "fatihportakal", count = 200)
df = timeline_df(tweets)

In [ ]:
?api.user_timeline

In [ ]:
?tweets

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#profil enleri

In [ ]:
df.sort_values("favorite_count", ascending = False).head()

In [ ]:
df.sort_values("retweet_count", ascending = False)[["text","retweet_count"]].iloc[0:3]

In [ ]:
df.sort_values("favorite_count", ascending = False)[["text","favorite_count"]].iloc[0:3]

## Retweet & Favori Sayılarının Dağılımlarının Çıkarılması

In [ ]:
df.head()

In [ ]:
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.distplot(df.favorite_count, kde = False, color = "blue");


In [ ]:
sns.distplot(df.retweet_count, color = "blue");
plt.xlim(-100, 5000)

## Tweet-Saat Dağılımı

In [ ]:
df.head()

In [ ]:
df["tweet_saati"] = df["created_at"].apply(lambda x: x.strftime("%H"))

In [ ]:
df.head()

In [ ]:
import pandas as pd
df["tweet_saati"] = pd.to_numeric(df["tweet_saati"])
df.info()

In [ ]:
sns.distplot(df["tweet_saati"], kde = False, color ="blue");

In [ ]:
df["gunler"] = df["created_at"].dt.weekday_name

In [ ]:
df.head()

In [ ]:
df.groupby("gunler").count()["id"]

In [ ]:
gun_freq = df.groupby("gunler").count()["id"]

In [ ]:
gun_freq.plot.bar(x = "gunler", y = "id")

## Tweet Atma Kaynaklarının Betimlenmesi

In [ ]:
kaynak_freq = df.groupby("source").count()["id"]

In [ ]:
kaynak_freq.plot.bar(x = "source", y = "id")

In [ ]:
df.groupby("source").count()["id"]

In [ ]:
df.groupby(["source","tweet_saati","gunler"])[["tweet_saati"]].count()

## Takipçi ve Arkadaşların Analizi

In [ ]:
user = api.get_user(id = "fatihportakal")

In [ ]:
for friend in user.friends():
    print(friend.screen_name)

In [ ]:
friends = user.friends()
followers = user.followers()

In [ ]:
def followers_df(takipci):
    import pandas as pd
    idler = [kisi.id for kisi  in takipci]
    df = pd.DataFrame(idler, columns = ["id"])
    
    df["created_at"] = [kisi.created_at for kisi in takipci]
    df["screen_name"] = [kisi.screen_name for kisi in takipci]
    df["location"] = [kisi.location for kisi in takipci]
    df["followers_count"] = [kisi.followers_count for kisi in takipci]
    df["statuses_count"] = [kisi.statuses_count for kisi in takipci]
    df["friends_count"] = [kisi.friends_count for kisi in takipci]
    df["favourites_count"] = [kisi.favourites_count for kisi in takipci]
    
    return df

In [ ]:
df = followers_df(followers)

In [ ]:
df.head()

## Takipçi Segmentasyonu

In [ ]:
df.head()

In [ ]:
df.index = df["screen_name"]

In [ ]:
df.head()

In [ ]:
s_data = df[["followers_count", "statuses_count"]]

In [ ]:
s_data.info()

In [ ]:
s_data

In [ ]:
s_data["followers_count"] = s_data["followers_count"] + 0.01
s_data["statuses_count"] = s_data["statuses_count"] + 0.01

In [ ]:
s_data

In [ ]:
6.01 * 1.01 

In [ ]:
4.01 * 39.01

In [ ]:
3 * 0.01

In [ ]:
s_data = s_data.apply(lambda x: (x-min(x))/(max(x)-min(x)))

In [ ]:
s_data["followers_count"] = s_data["followers_count"] + 0.01
s_data["statuses_count"] = s_data["statuses_count"] + 0.01

In [ ]:
s_data.head()

In [ ]:
skor = s_data["followers_count"] * s_data["statuses_count"]

In [ ]:
skor

In [ ]:
skor.sort_values(ascending = False)

In [ ]:
skor[skor > skor.median() + skor.std()/len(skor)].sort_values(ascending = False)

In [ ]:
skor.std()

In [ ]:
skor.median()

In [ ]:
s_data["skor"] = skor

In [ ]:
s_data

In [ ]:
import numpy as np
s_data["segment"] = np.where(s_data["skor"] >= skor.median() + 
                             skor.std()/len(skor), "A","B")

In [ ]:
s_data

# Hashtag Analizi

In [ ]:
api.trends_available()

In [ ]:
def ulke_kodlari():
    places = api.trends_available()
    all_woeids = {place['name'].lower(): place['woeid'] for place in places}
    return all_woeids

In [ ]:
ulke_kodlari()

In [ ]:
def ulke_woeid(ulke_adi):
    ulke_adi = ulke_adi.lower()
    trends = api.trends_available()
    all_woeids = ulke_kodlari()
    return all_woeids[ulke_adi]
    

In [ ]:
23424969

In [ ]:
ulke_woeid("turkey")

In [ ]:
trendler = api.trends_place(id = 23424969)

In [ ]:
import json
print(json.dumps(trendler, indent = 4))

In [ ]:
turkiye = api.trends_place(id = 23424969)
trendler = turkiye[0]["trends"]

In [ ]:
for i in trendler:
    print(i["name"])

## Hashtag'den Veri Çekmek

In [ ]:
tweetler = api.search(q = "#pazartesi", 
                      lang = "tr", 
                      result_type = "recent", 
                     count = 1000)

In [ ]:
def hashtag_df(tweetler):
    import pandas as pd
    id_list = [tweet.id for tweet  in tweetler]
    df = pd.DataFrame(id_list, columns = ["id"])
    
    df["text"] = [tweet.text for tweet in tweetler]
    df["created_at"] = [tweet.created_at for tweet in tweetler]
    df["retweeted"] = [tweet.retweeted for tweet in tweetler]
    df["retweet_count"] = [tweet.retweet_count for tweet in tweetler]
    df["user_screen_name"] = [tweet.author.screen_name for tweet in tweetler]
    df["user_followers_count"] = [tweet.author.followers_count for tweet in tweetler]
    df["user_location"] = [tweet.author.location for tweet in tweetler]
    df["Hashtags"] = [tweet.entities.get('hashtags') for tweet in tweetler]
    
    return df

In [ ]:
df = hashtag_df(tweetler)

In [ ]:
df.shape

In [ ]:
df.head()

## Hashtag Betimlemek

In [ ]:
#essiz katılımcı

In [ ]:
df.head()

In [ ]:
df["user_screen_name"].unique().size

In [ ]:
df.groupby("user_screen_name")["id"].count().sum()

In [ ]:
#en cok katkı sağlayanlar

In [ ]:
df.groupby("user_screen_name").count()["id"].sort_values(ascending = False)[0:5]

In [ ]:
#ozgun katkı

In [ ]:
df.head()

In [ ]:
df["text"].str.startswith("RT").head(10)

In [ ]:
df[~df["text"].str.startswith("RT")].count()["id"] / len(df)

In [ ]:
df[~df["text"].str.startswith("RT")].count()["id"] / df[df["text"].str.startswith("RT")].count()["id"]

In [ ]:
## en cok fav-retweet

In [ ]:
df.sort_values("retweet_count", ascending = False).head()

## Tweet-Saat Dağılımı

In [ ]:
?api.search

In [ ]:
tweetler = api.search(q = "#datascience", lang = "en", count = 5000)

In [ ]:
def hashtag_df(tweetler):
    import pandas as pd
    id_list = [tweet.id for tweet  in tweetler]
    df = pd.DataFrame(id_list, columns = ["id"])
    
    df["text"] = [tweet.text for tweet in tweetler]
    df["created_at"] = [tweet.created_at for tweet in tweetler]
    df["retweeted"] = [tweet.retweeted for tweet in tweetler]
    df["retweet_count"] = [tweet.retweet_count for tweet in tweetler]
    df["source"] = [tweet.source for tweet in tweetler]
    df["user_screen_name"] = [tweet.author.screen_name for tweet in tweetler]
    df["user_followers_count"] = [tweet.author.followers_count for tweet in tweetler]
    df["user_location"] = [tweet.author.location for tweet in tweetler]
    df["Hashtags"] = [tweet.entities.get('hashtags') for tweet in tweetler]
    
    return df

In [ ]:
df = hashtag_df(tweetler)

In [ ]:
df.shape

In [ ]:
df["tweet_saati"] = df["created_at"].apply(lambda x: x.strftime("%H"))

In [ ]:
df.head()

In [ ]:
import pandas as pd
df["tweet_saati"] = pd.to_numeric(df["tweet_saati"])
df.info()

In [ ]:
sns.distplot(df["tweet_saati"], kde = False, color ="blue");

In [ ]:
df["gunler"] = df["created_at"].dt.weekday_name

In [ ]:
df.head()

In [ ]:
df.groupby("gunler").count()["id"]

In [ ]:
gun_freq = df.groupby("gunler").count()["id"]

In [ ]:
gun_freq.plot.bar(x = "gunler", y = "id")

## Tweet Atma Kaynaklarının Betimlenmesi

In [ ]:
kaynak_freq = df.groupby("source").count()["id"]

In [ ]:
kaynak_freq.plot.bar(x = "source", y = "id")

In [ ]:
df.groupby("source").count()["id"]

# Twitter Text Mining

In [ ]:
df = hashtag_df(tweetler)

#buyuk-kucuk donusumu
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')

#sayılar
df['text'] = df['text'].str.replace('\d','')

#stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

#lemmi
from textblob import Word
#nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 

#noktalama işaretleri
df['text'] = df['text'].str.replace('rt','')

In [ ]:
df["text"]

In [ ]:
freq_df = df["text"].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis =0).reset_index()


In [ ]:
freq_df.columns = ["kelimeler","frekanslar"]

In [ ]:
freq_df.shape

In [ ]:
freq_df[freq_df.frekanslar > freq_df.frekanslar.mean() + 
        freq_df.frekanslar.std()]

In [ ]:
a = freq_df[freq_df.frekanslar > freq_df.frekanslar.mean() + 
        freq_df.frekanslar.std()]

In [ ]:
a.plot.bar(x = "kelimeler",y = "frekanslar");

## Word cloud

In [ ]:
#!pip install wordcloud

In [ ]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
text = " ".join(i for i in df.text)

In [ ]:
text

In [ ]:
wordcloud = WordCloud(background_color = "white").generate(text)
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
vbo_mask = np.array(Image.open("VBO.jpg"))

In [ ]:
wc = WordCloud(background_color = "white", 
               max_words = 1000, 
               mask = vbo_mask,
              contour_width = 3, 
              contour_color = "firebrick")

wc.generate(text)
plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation = "bilinear")
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

# Twitter Sentiment Analizi

In [ ]:
df

In [ ]:
from textblob import TextBlob

In [ ]:
def sentiment_skorla(df):

    text = df["text"]

    for i in range(0,len(text)):
        textB = TextBlob(text[i])
        sentiment_skoru = textB.sentiment.polarity
        df.set_value(i, 'sentiment_skoru', sentiment_skoru)
        
        if sentiment_skoru <0.00:
            duygu_sinifi = 'Negatif'
            df.set_value(i, 'duygu_sinifi', duygu_sinifi )

        elif sentiment_skoru >0.00:
            duygu_sinifi = 'Pozitif'
            df.set_value(i, 'duygu_sinifi', duygu_sinifi )

        else:
            duygu_sinifi = 'Notr'
            df.set_value(i, 'duygu_sinifi', duygu_sinifi )
            
    return df 

In [ ]:
df.head()

In [ ]:
sentiment_skorla(df)

In [ ]:
df.groupby("duygu_sinifi").count()["id"]

In [ ]:
duygu_freq = df.groupby("duygu_sinifi").count()["id"]

In [ ]:
duygu_freq.plot.bar(x = "duygu_sinifi",y = "id");

In [ ]:
tweetler = api.search(q = "#apple", lang = "en")

In [ ]:
df = hashtag_df(tweetler)

In [ ]:
#buyuk-kucuk donusumu
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')

#sayılar
df['text'] = df['text'].str.replace('\d','')

#stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

#lemmi
from textblob import Word
#nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 

#noktalama işaretleri
df['text'] = df['text'].str.replace('rt','')

In [ ]:
sentiment_skorla(df)

In [ ]:
duygu_freq = df.groupby("duygu_sinifi").count()["id"]

In [ ]:
duygu_freq.plot.bar(x = "duygu_sinifi",y = "id");